In [ ]:
import tensorflow as tf
from tensorflow import convert_to_tensor, string
from tensorflow.keras.layers import TextVectorization, Embedding, Layer
from tensorflow.data import Dataset
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
output_sequence_length = 5
vocab_size = 10
sentences = [["I am a robot"], ["you too robot"]]
sentence_data = Dataset.from_tensor_slices(sentences)
vectorize_layer = TextVectorization(output_sequence_length=output_sequence_length, max_tokens=vocab_size)

vectorize_layer.adapt(sentence_data)

word_tensors = convert_to_tensor(sentences, dtype=tf.string)

vectorized_words = vectorize_layer(word_tensors)

print(vectorize_layer.get_vocabulary())
print(vectorized_words)

In [ ]:
output_length = 6
word_embedding_layer = Embedding(vocab_size, output_length)
embedded_words = word_embedding_layer(vectorized_words)
print(embedded_words)

In [ ]:
position_embedding_layer = Embedding(output_sequence_length, output_length)
position_indices = tf.range(output_sequence_length)
print(position_indices)
embedded_indices = position_embedding_layer(position_indices)
print(embedded_indices)

In [ ]:
final_output_embedding = embedded_words + embedded_indices
print("Final output: ", final_output_embedding)